# Proximal Causal Inference Synthetic Simulations

In this set of experiments, we attempt to simulate text-based proximal causal inference when two pieces of text data are independent and representative of the underlying distribution of text data. Below are the steps of the experiment:

1. Make a training dataset DTrain. For each row of data in DTrain, generate two realizations of X1, X2, X3, and X4. 
2. Generate X = [mean(X11, X12), mean(X21, X22), mean(X31, X32), mean(X41, X42)]. Train a single linear logistic regression on this data to predict U. This simulates training a zero-shot classifier on some broad background/training data.
3. Make a dataset that simulates inference time DInference. For each row of data, generate two realizations of X1, X2, X3, and X4. These two realizations of X1 through X4 are independent by definition and represent independent realizations of the oracle U.
4. Apply the previously trained 'zero-shot model' to obtain Z and W from the two realizations of X1, X2, X3, and X4.
5. Proceed with proximal pipeline as usual.

In [1]:
import pandas as pd
import numpy as np
from scipy.special import expit
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pickle
from create_latex_table import *

import warnings

# Suppress perfect separation warnings
# we are supressing these warnings because perfect separation is expected when ignore
# Gotcha 3 in our P1M experiments
warnings.filterwarnings("ignore")

np.random.seed(7)
size = 20000

Code implementing future helper functions.

In [2]:
def proximal_find_ace(A, Y, W, Z, covariates, data):
    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=[Y]), data[Y], test_size=0.5)

    # subset the features to just A, Z, and covariates and the outcome to W
    model1_features = X_train[[A]+[Z]+covariates]
    model1_outcome = X_train[W]

    # if there is a high amount of class imbalance, rebalance the class weights
    if np.mean(model1_outcome) < 0.2 or np.mean(model1_outcome) > 0.8:
        model1 = LogisticRegression(class_weight="balanced", penalty=None)
    else:
        model1 = LogisticRegression(penalty=None)

    model1.fit(model1_features, model1_outcome)

    # make predictions on the probability
    What = model1.predict_proba(X_test[[A]+[Z]+covariates])[:, 1]
    # print(np.mean(What))

    X_test["What"] = What

    # train a linear regression for the second stage of the estimation strategy
    model2_features = X_test[[A]+["What"]+covariates]
    model2_outcome = y_test

    model2 = LinearRegression()
    model2.fit(model2_features, model2_outcome)
    
    return model2.coef_[0]

def compute_confidence_intervals(A, Y, W, Z, covariates, data, num_bootstraps=200, alpha=0.05):
    """
    Compute confidence intervals for proximal causal inference via bootstrap
    
    Returns tuple (q_low, q_up) for the lower and upper quantiles of the confidence interval.
    """
    
    Ql = alpha/2
    Qu = 1 - alpha/2
    # two lists for the two indexes of output
    estimates = []
    
    for i in range(num_bootstraps):
        
        # resample the data with replacement
        data_sampled = data.sample(len(data), replace=True)
        data_sampled.reset_index(drop=True, inplace=True)
        
        # add estimate from resampled data
        output = proximal_find_ace(A, Y, W, Z, covariates, data_sampled)
        estimates.append(output)

    # calculate the quantiles
    quantiles = np.quantile(estimates, q=[Ql, Qu])
    q_low = quantiles[0]
    q_up = quantiles[1]
    
    return (q_low, q_up)

def odds_ratio(X, Y, Z, data):
    features = data[[Y]+Z]
    outcome = data[X]

    if np.mean(outcome) < 0.2 or np.mean(outcome) > 0.8:
        model = LogisticRegression(class_weight="balanced", penalty=None)
    else:
        model = LogisticRegression(penalty=None)
    model.fit(features, outcome)

    return np.exp(model.coef_[0][0])

def odds_ratio_confidence_intervals(X, Y, Z, data, num_bootstraps=200, alpha=0.05):
    """
    Get bootstrap confidence intervals for the value of the odds ratio
    """

    Ql = alpha/2
    Qu = 1 - alpha/2
    # two lists for the two indexes of output
    estimates = []
    
    for i in range(num_bootstraps):
        
        # resample the data with replacement
        data_sampled = data.sample(len(data), replace=True)
        data_sampled.reset_index(drop=True, inplace=True)
        
        # add estimate from resampled data
        output = odds_ratio(X, Y, Z, data_sampled)
        estimates.append(output)

    # calculate the quantiles
    quantiles = np.quantile(estimates, q=[Ql, Qu])
    q_low = quantiles[0]
    q_up = quantiles[1]
    
    return (q_low, q_up)

Create a list to save results into and eventaully into a pickle file.

In [3]:
save = []

Define the data generating process.

In [4]:
def generate_data():
    U = np.random.binomial(1, 0.48, size)

    # create a baseline confounder
    C = np.random.normal(0, 1, size)
    C_coefficient = 3

    X11 = np.random.normal(0, 1, size) + 1.95*U + C_coefficient*C
    X12 = np.random.normal(0, 1, size) + 1.95*U + C_coefficient*C

    # make sure that X2 is some non-linear function
    X21 = np.random.normal(0, 1, size) + np.exp(X11) + U + C_coefficient*C
    X22 = np.random.normal(0, 1, size) + np.exp(X12) + U + C_coefficient*C

    X31 = np.random.normal(0, 1, size) + 1.25*U + C_coefficient*C
    X32 = np.random.normal(0, 1, size) + 1.25*U + C_coefficient*C

    # make sure that X4 is some non-linear function
    X41 = np.random.normal(0, 1, size) + X31**2 + 0.5*X31**3 + U + C_coefficient*C
    X42 = np.random.normal(0, 1, size) + X32**2 + 0.5*X32**3 + U + C_coefficient*C

    A = np.random.binomial(1, expit(0.8*U+C-0.3), size)

    Y = np.random.normal(0, 1, size) + 1.3*A + 0.8*U + 1*C

    data = pd.DataFrame({"U": U, "X11": X11, "X21": X21, "X31": X31, "X41": X41, "X12": X12, "X22": X22, "X32": X32, "X42": X42,
                         "A": A, "Y": Y, "C": C})

    return data

Generate training dataset.

In [5]:
Dtrain = generate_data()

# simulate the creation of a 'broad zero-shot predictor'
Dtrain['X1'] = (Dtrain['X11']+Dtrain['X12'])/2
Dtrain['X2'] = (Dtrain['X21']+Dtrain['X22'])/2
Dtrain['X3'] = (Dtrain['X31']+Dtrain['X32'])/2
Dtrain['X4'] = (Dtrain['X41']+Dtrain['X42'])/2

Train a linear logistic regressor to simulate a zero-shot classifier.

In [6]:
features = Dtrain[['X1', 'X2', 'X3', 'X4']]
outcome = Dtrain['U']

zero_shot_model = LogisticRegression(penalty=None)
zero_shot_model.fit(features, outcome)

LogisticRegression(penalty=None)

Generate the inference dataset, and generate Z and W by using the two realizations of the covariates.

In [7]:
DInference = generate_data()

# create the two realizations of the covariates to generate Z and W
realization1 = pd.DataFrame()
realization1['X1'] = DInference['X11']
realization1['X2'] = DInference['X21']
realization1['X3'] = DInference['X31']
realization1['X4'] = DInference['X41']

W = zero_shot_model.predict(realization1[['X1', 'X2', 'X3', 'X4']])

realization2 = pd.DataFrame()
realization2['X1'] = DInference['X12']
realization2['X2'] = DInference['X22']
realization2['X3'] = DInference['X32']
realization2['X4'] = DInference['X42']

Z = zero_shot_model.predict(realization2[['X1', 'X2', 'X3', 'X4']])

DInference['Z'] = Z
DInference['W'] = W

Verify odds ratio values between W and Z.

In [8]:
print("odds ratio, nominal:", odds_ratio("W", "Z", [], DInference))
print("odds ratio, condition C:", odds_ratio("W", "Z", ["C"], DInference))
print("odds ratio, condition C and U:", odds_ratio("W", "Z", ["C", "U"], DInference))
or_ci = odds_ratio_confidence_intervals("W", "Z", ["C"], DInference)
print("odds ratio confidence interval:", or_ci)

odds ratio, nominal: 2.036667012105297
odds ratio, condition C: 1.3819210904788075
odds ratio, condition C and U: 1.0004344685417592
odds ratio confidence interval: (1.350918677956896, 1.4159820364036846)


Proceed with the causal inference pipeline as normal.

In [9]:
est_ace = proximal_find_ace("A", "Y", "W", "Z", ["C"], DInference)
bias = abs(1.3-est_ace)
ace_ci = compute_confidence_intervals("A", "Y", "W", "Z", ["C"], DInference)
ci_cov = f'{{\\bf Yes}}'

print(est_ace)
print(ace_ci)

1.3035777535463435
(1.2092251113030785, 1.3939507481471058)


In [10]:
save.append({
    'pipeline': 'P1M',
    'or_ci_low': or_ci[0],
    'or_ci_high': or_ci[1],
    'est_ace': est_ace,
    'bias': bias,
    'ace_ci': ace_ci,
    'ci_cov': ci_cov
})

Generate the proxies from the same realization of data, this violates P1 on purpose and should produce biased results.

*Note:* The following procedure by definition will generate the same proxies. This explains why the proximal pipleine is throwing perfect separation errors.

In [11]:
W = zero_shot_model.predict(realization1[['X1', 'X2', 'X3', 'X4']])

Z = zero_shot_model.predict(realization1[['X1', 'X2', 'X3', 'X4']])

DInference['Z'] = Z
DInference['W'] = W

Verify odds ratio values.

In [12]:
print("odds ratio, nominal:", odds_ratio("W", "Z", [], DInference))
print("odds ratio, condition C:", odds_ratio("W", "Z", ["C"], DInference))
print("odds ratio, condition C and U:", odds_ratio("W", "Z", ["C", "U"], DInference))
or_ci = odds_ratio_confidence_intervals("W", "Z", ["C"], DInference)
print("odds ratio confidence interval:", or_ci)

odds ratio, nominal: 6226206818464955.0
odds ratio, condition C: 1.4934648023714422e+16
odds ratio, condition C and U: 1.4618008444686512e+16
odds ratio confidence interval: (1.3192862015421102e+16, 1.615047285264014e+16)


In [13]:
est_ace = proximal_find_ace("A", "Y", "W", "Z", ['C'], DInference)
bias = abs(1.3-est_ace)
ace_ci = compute_confidence_intervals("A", "Y", "W", "Z", ['C'], DInference)
ci_cov = f'No'

print(est_ace)
print(ace_ci)

1.4304801592918137
(1.4045551394655196, 1.4953321912501123)


In [14]:
save.append({
    'pipeline': 'P1M, same',
    'or_ci_low': or_ci[0],
    'or_ci_high': or_ci[1],
    'est_ace': est_ace,
    'bias': bias,
    'ace_ci': ace_ci,
    'ci_cov': ci_cov
})

## Part 2

Use the zero-shot predictor to generate a proxy W and a simple heuristic based predictor to generate the proxy Z. Then, proceed with proximal causal inference as normal.

In [15]:
DInference = generate_data()

# create the first realization of the covariates to generate W
realization1 = pd.DataFrame()
realization1['X1'] = DInference['X11']
realization1['X2'] = DInference['X21']
realization1['X3'] = DInference['X31']
realization1['X4'] = DInference['X41']

W = zero_shot_model.predict(realization1[['X1', 'X2', 'X3', 'X4']])

# use a simple heuristic for the second realization of the covariates to generate the proxy Z
# (this heuristic attains roughly accuracy 0.84)
Z = []
for i in range(size):
    if DInference['X12'][i] > 1.1:
        Z.append(1)
    else:
        Z.append(0)

DInference['W'] = W
DInference['Z'] = Z

Verify odds ratio values between W and Z.

In [16]:
print("odds ratio, nominal:", odds_ratio("W", "Z", [], DInference))
print("odds ratio, condition C:", odds_ratio("W", "Z", ["C"], DInference))
print("odds ratio, condition C and U:", odds_ratio("W", "Z", ["C", "U"], DInference))
or_ci = odds_ratio_confidence_intervals("W", "Z", ["C"], DInference)
print("odds ratio confidence intervals:", or_ci)

odds ratio, nominal: 3.63752559165995
odds ratio, condition C: 1.880269157051084
odds ratio, condition C and U: 1.0363406022327513
odds ratio confidence intervals: (1.820170440932906, 1.9363521194561315)


Proceed with proximal causal inference pipeline.

In [17]:
est_ace = proximal_find_ace("A", "Y", "W", "Z", ["C"], DInference)
bias = abs(1.3-est_ace)
ace_ci = compute_confidence_intervals("A", "Y", "W", "Z", ["C"], DInference)
ci_cov = f'{{\\bf Yes}}'

print(est_ace)
print(ace_ci)

1.3431202293299802
(1.2731587609051653, 1.4246635248211406)


In [18]:
save.append({
    'pipeline': 'P2M',
    'or_ci_low': or_ci[0],
    'or_ci_high': or_ci[1],
    'est_ace': est_ace,
    'bias': bias,
    'ace_ci': ace_ci,
    'ci_cov': ci_cov
})

Generate the proxies from the same realization of data, this violates P1 on purpose and should produce biased results.

In [19]:
W = zero_shot_model.predict(realization1[['X1', 'X2', 'X3', 'X4']])

# use a simple heuristic on the same realization of the covariates to generate the proxy Z
# (this heuristic attains roughly accuracy 0.84)
Z = []
for i in range(size):
    if DInference['X11'][i] > 1.1:
        Z.append(1)
    else:
        Z.append(0)

DInference['W'] = W
DInference['Z'] = Z

Verify odds ratio values.

In [20]:
print("odds ratio, nominal:", odds_ratio("W", "Z", [], DInference))
print("odds ratio, condition C:", odds_ratio("W", "Z", ["C"], DInference))
print("odds ratio, condition C and U:", odds_ratio("W", "Z", ["C", "U"], DInference))
or_ci = odds_ratio_confidence_intervals("W", "Z", ["C"], DInference)
print("odds ratio confidence interval:", or_ci)

odds ratio, nominal: 7.45302627844804
odds ratio, condition C: 8.162085461375293
odds ratio, condition C and U: 5.643790451130861
odds ratio confidence interval: (7.901714409322671, 8.40806602828209)


In [21]:
est_ace = proximal_find_ace("A", "Y", "W", "Z", ["C"], DInference)
bias = abs(1.3-est_ace)
ace_ci = compute_confidence_intervals("A", "Y", "W", "Z", ["C"], DInference)
ci_cov = f'No'

print(est_ace)
print(ace_ci)

1.4074315971024316
(1.3764490464840136, 1.4786932779637072)


In [22]:
save.append({
    'pipeline': 'P2M, same',
    'or_ci_low': or_ci[0],
    'or_ci_high': or_ci[1],
    'est_ace': est_ace,
    'bias': bias,
    'ace_ci': ace_ci,
    'ci_cov': ci_cov
})

In [23]:
pickle.dump(save, open('fully_synthetic_save.p', 'wb'))

In [26]:
print(create_table('fully_synthetic_save.p'))

		P1M & {\color{blue} $(1.35, 1.42)^\checkmark$} & $1.304$ & $0.004$ & $(1.209, 1.394)$ & {\bf Yes} \\
		P1M, same & {\color{red} $(1.32e+16, 1.62e+16)$} & $1.430$ & $0.130$ & $(1.405, 1.495)$ & No \\
		P2M & {\color{blue} $(1.82, 1.94)^\checkmark$} & $1.343$ & $0.043$ & $(1.273, 1.425)$ & {\bf Yes} \\
		P2M, same & {\color{red} $(7.9, 8.41)$} & $1.407$ & $0.107$ & $(1.376, 1.479)$ & No \\

